

<center><h1> CSE 4693/6693 Intro to Machine Learning </h1></center>

## Project name: GAN Channel Modeling (Group 4)

<div style="float: left; margin-right: 20px;">

| Name | NetID |
|:-----|:------|
| Joshua Moore | jjm702 |
| Tirian Judy | tkj105 |
| Claire Johnson | kj1289 |
| Aayam Raj Shakya | as5160 |

</div>

In [ ]:
#Imports & some predefines
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import os
import glob
import pandas as pd
import pickle

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Use cuda if possible; otherwise eat some threads on the CPU

In [ ]:
#Read one of the data files #TODO: Read all csvs

features = ['center_frequency', 'distance', 'h_dist', 'v_dist', 'avgPower', 'avgSnr',
            'freq_offset', 'avg_pl', 'aod_theata', 'aoa_theta', 'aoa_phi',
            'pitch', 'yaw', 'roll', 'vel_x', 'vel_y', 'vel_z', 'speed', 'avg_pl_rolling', 'avg_pl_ewma']

df = pd.read_csv("/content/2023-12-15_15_41-results.csv", usecols = features)


#Need to ensure only valid data goes beyond here #TODO

data = df[features].values#[:100] Limit values be needed - ask

# Function to create sequences
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(data, seq_length)

# Convert data to PyTorch tensors - no filter currently, reproduce everything
trainX = torch.tensor(X, dtype=torch.float32)
trainY = torch.tensor(y, dtype=torch.float32)


In [ ]:
#Copy of Aayam's LSMT class - might need to be altered?
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        # If hidden and cell states are not provided, initialize them as zeros
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)

        # Forward pass through LSTM
        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Selecting the last output
        return out, hn, cn


In [ ]:
# Initialize model, loss, and optimizer
model = LSTMModel(input_dim=len(features), hidden_dim=50, layer_dim=1, output_dim=len(features))
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
#Training loop - LSTM
num_epochs = 4000
h0, c0 = None, None  # Initialize hidden and cell states

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs, h0, c0 = model(trainX, h0, c0)

    # Compute loss
    loss = criterion(outputs, trainY)
    loss.backward()
    optimizer.step()

    # Detach hidden and cell states to prevent backpropagation through the entire sequence
    h0 = h0.detach()
    c0 = c0.detach()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
#TODO - validate outputs now that it should be for multiple values not just SNR

In [ ]:
#Build the Generator
class Generator(nn.Module):

    def __init__(self, inputSize=6, outputSize =(48,48)): #LSTM model should produce a vector of 6 terms; Output an array of 48H * 48W
        super(Generator, self).__init__() #We should be fine using Pytorch base model code

        self.outputSize = outputSize #Need to remember the output size for the forward function

        self.model = nn.Sequential(nn.Linear(inputSize, 128), #mapping input vector to 128 neurons; 
                                   nn.LeakyReLU(0.2, inplace=True), #Using inplace=True to save memory + it seems to be standard practice
                                   #start hidden layers
                                   nn.Linear(128,256), #Hidden Layer 1; Upscaling by factor of 2
                                   nn.BatchNorm1d(256), #Normalization
                                   nn.LeakyReLU(0.2, inplace=True), 
                                   nn.Linear(256, 512), #Hidden Layer 2; Upsacling by factor of 2; Last hidden layer for now
                                   nn.BatchNorm1d(512), #Normalization
                                   nn.LeakyReLu(0.2, inplace=True),
                                   nn.Linear(512, outputSize[0] * outputSize[1]), #Output layer
                                   nn.Tanh() #Normalization for output (-1, 1)
                                   )
    
    #Passes data into first layer & executes sequentially based upon params from self.model()
    def forward(self, noise):
        genImage =  self.model(noise) #Grab output tensor
        return genImage.view(-1, 1, self.outputSize[0], self.outputSize[1]) #Should produce a grey scale image using the view function. Args: Batchsize(-1 means figure it out for me), numChannels, height, width

In [ ]:
#Build the Discriminator
class Discriminator(nn.Module):

    def __init__(self, inputSize=(48,48)): #Simple classifier returns [0,1]; 1 real
        super(Discriminator, self).__init__() #Use the base Pytorch discriminator

        self.inputSize = inputSize #Need to remember the input size for the forward function

        self.model = nn.Sequential(nn.Linear(inputSize[0] * inputSize[1], 512), #mapping input vector to 512 neurons
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(512, 256), #Hidden Layer 1; Downscaling by factor of 2
                                   nn.LeakyReLU(0.2, inplace=True),
                                   nn.Linear(256, 1), #Output layer - Map 256 neurons into the probability of being real
                                   nn.Sigmoid() #Turn the output into [0,1]
        )
    
    def forward(self, input):
        flatInput = input.view(input.size(0), -1) #Flatten the input to 1D
        return self.model(flatInput)

In [ ]:
import torch.utils.data import DataLoader

#Hyper params
GANepochs = 1000
GANlr = 0.0002 #Should probably be low

#Labels - DO NOT USE 1 or 0, it will be to ridged, smooth it by ~.025 - .2
realLabel = 0.9
fakeLabel = 0.1

#Loss function
losFunc = nn.BCELoss()

#Creating the Discriminator
discrim = Discriminator().to(DEVICE)
dOptimizer = torch.optim.Adam(discrim.parameters(), lr=0.0002, betas=(0.5, 0.999)) #Lower B1 since models will fight eachother

#Creating the Generator
generator = Generator().to(DEVICE)
gOptimizer = torch.optim.Adam(generator.parameter(), lr=0.0002, betas=(0.5, 0.999)) #Lower B1 since models will fight eachother




#load data somehow
#It would be visualizations of the data no?
dataloaded = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=None, num_workers=0, pin_memory=False, tiemout=0)

#static noise
fixedNoise = torch.randn(10, len(features)).to(DEVICE)

for epoch in range(GANepochs):
    #Discriminator goes to the gym
    for i, realSpectrogram in enumerate(dataloaded):
        realSpectrogram = realSpectrogram.to(DEVICE) #Shove it on the GPU
        
        real = torch.full((len(realSpectrogram), 1), realLabel, device=DEVICE)
        fake = torch.full((len(realSpectrogram), 1), fakeLabel, device=DEVICE)

        #Discrim on the real images
        dOptimizer.zero_grad() #0 gradient
        output = discrim()



